In [2]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [3]:
def read_sequence(sequence_path, to_str=False):
    seqs = []
    with open(sequence_path, 'r') as f:
        for record in SeqIO.parse(f, 'fasta'):
            record = SeqRecord(record.seq.upper(), id=record.id, description=record.description)
            seqs.append(record)
    if to_str:
        return [str(seq.seq) for seq in seqs]
    else:
        return seqs

In [5]:
# 加载cDNA序列
cdna_path = './Homo_sapiens.GRCh38.cdna.all.fa'
cdna_seq = read_sequence(cdna_path)
print(len(cdna_seq))

191887


In [6]:
import re
import pandas as pd

In [7]:
def get_IDs(line):
    gene = None
    trans = line.split(" ")[0]
    syml = None
    chrom = None
    trans_type = None
    gene_p = r'gene:(.*?)$'
    syml_p = r'gene_symbol:(.*?)$'
    chrom_p = r'chromosome:(.*?)$'
    trans_p = r'transcript_biotype:(.*?)$'
    for elem in line.split(" "):
        gene_m = re.search(gene_p, elem)
        syml_m = re.search(syml_p, elem)
        chrom_m = re.search(chrom_p, elem)
        trans_m = re.search(trans_p, elem)
        if gene_m:
            gene = gene_m.group(1)
        elif syml_m:
            syml = syml_m.group(1)
        elif chrom_m:
            chrom = chrom_m.group(1)
            c = chrom.split(":")
            chrom = f'{c[1]}:{c[2]}-{c[3]}:{c[4]}'
        elif trans_m:
            trans_type = trans_m.group(1)
    return [trans, gene, syml, chrom, trans_type]

def getTransformat(cdna_seq):
    anno = []
    for record in cdna_seq:
        des = record.description
        fmt = get_IDs(des)
        fmt.append(len(str(record.seq)))
        anno.append(fmt)
    df = pd.DataFrame(anno, columns=['Transcript_ID', 'Gene_ID', 'Gene_Symbol', 'Gene_Position', 'Transcript_Biotype', 'Transcript_Length'])
    df.drop_duplicates(inplace=True)
    df.sort_values(by=["Gene_Symbol", "Transcript_Length", "Transcript_ID", "Gene_Position"], ascending=[True, False, True, False], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [8]:
# 将cDNA中的信息提取出来
cdna_anno = getTransformat(cdna_seq)
cdna_anno

,Transcript_ID,Gene_ID,Gene_Symbol,Gene_Position,Transcript_Biotype,Transcript_Length
0,ENST00000620796.1,ENSG00000275780.1,5S_rRNA,CHR_HSCHR17_2_CTG5:45327366-45327497:-1,rRNA_pseudogene,132
1,ENST00000637405.1,ENSG00000283454.1,5S_rRNA,CHR_HSCHR17_1_CTG5:45327366-45327497:-1,rRNA_pseudogene,132
2,ENST00000648701.1,ENSG00000285609.1,5S_rRNA,1:182944365-182944490:-1,rRNA_pseudogene,126
3,ENST00000648813.1,ENSG00000285674.1,5S_rRNA,19:7886977-7887096:-1,rRNA_pseudogene,120
4,ENST00000647846.1,ENSG00000285776.1,5S_rRNA,X:69672479-69672597:1,rRNA_pseudogene,119
...,...,...,...,...,...,...
191882,ENST00000433749.5,ENSG00000036549.13,ZZZ3,1:77633182-77682675:-1,protein_coding,603
191883,ENST00000463166.5,ENSG00000036549.13,ZZZ3,1:77633312-77683403:-1,processed_transcript,587
191884,ENST00000474746.1,ENSG00000036549.13,ZZZ3,1:77581818-77585059:-1,processed_transcript,572
191885,ENST00000414381.5,ENSG00000036549.13,ZZZ3,1:77633246-77683374:-1,protein_coding,561


In [9]:
# 将靶标cDNA序列选择出来
target_seq = [seq for seq in cdna_seq if 'VEGFA' in seq.description]
target_anno = getTransformat(target_seq)
print(len(target_seq))
print(target_anno)

26
         Transcript_ID             Gene_ID Gene_Symbol          Gene_Position  \
0    ENST00000480614.1  ENSG00000112715.24       VEGFA  6:43773172-43786484:1   
1    ENST00000372067.7  ENSG00000112715.24       VEGFA  6:43770213-43786487:1   
2    ENST00000672860.2  ENSG00000112715.24       VEGFA  6:43770213-43786487:1   
3    ENST00000372064.8  ENSG00000112715.24       VEGFA  6:43770707-43786487:1   
4    ENST00000372077.8  ENSG00000112715.24       VEGFA  6:43770782-43786487:1   
5    ENST00000497139.5  ENSG00000112715.24       VEGFA  6:43777720-43786439:1   
6    ENST00000476772.5  ENSG00000112715.24       VEGFA  6:43770184-43778764:1   
7    ENST00000425836.6  ENSG00000112715.24       VEGFA  6:43770707-43784949:1   
8    ENST00000372055.8  ENSG00000112715.24       VEGFA  6:43770707-43784609:1   
9    ENST00000520948.5  ENSG00000112715.24       VEGFA  6:43771083-43784949:1   
10   ENST00000413642.7  ENSG00000112715.24       VEGFA  6:43770707-43784589:1   
11   ENST00000482630.6  E

In [10]:
# 加载sgRNA序列文件
sgRNA_path = './sgRNA_sequence.fasta'
sgRNA_seq = read_sequence(sgRNA_path)
print(len(sgRNA_seq))

3


In [11]:
from Bio import pairwise2
from Bio.Seq import Seq
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor


In [12]:
def get_cigar_string(alignments, start, end):
    cigar = ''
    seq1 = alignments[0]
    seq2 = alignments[1]
    for i in range(start, end):
        if seq1[i] == seq2[i]:
            cigar += '|'
        elif seq1[i] == '-':
            cigar += 'I'
        elif seq2[i] == '-':
            cigar += 'D'
        else:
            cigar += 'X'
    return cigar


def pairwise_local_align(seq1, seq2, max_mismatches=None, min_consecutive_matches=None):
    try:
        alignments = pairwise2.align.localms(seq1.seq, seq2.seq, 2, -1, -8, -0.5, one_alignment_only=True, gap_char='-', force_generic=True)
        cigar = get_cigar_string(alignments[0], alignments[0].start, alignments[0].end)
        matches = cigar.count('|')
        mismatches = cigar.count('X') + cigar.count('I') + cigar.count('D')
        start = alignments[0].start
        end = alignments[0].end
        max_consecutive_matches = max([len(x) for x in re.split('[DXI]', cigar)])
        if (max_mismatches and mismatches > max_mismatches) or (min_consecutive_matches and max_consecutive_matches < min_consecutive_matches):
            return None
        else:
            return {'matches': matches, 'mismatches': mismatches, 'start': start, 'end': end,
                    'max_consecutive_matches': max_consecutive_matches, 
                    'seq1':alignments[0][0][start:end], 'seq2':alignments[0][1][start:end], 'cigar':cigar}
    except Exception as e:
        print(e)
        return None
    
def reverse_complement(seq):
    if isinstance(seq, str):
        seq = seq.translate(str.maketrans("ATCG", "TAGC"))[::-1]
    else:
        seq.seq = seq.seq.reverse_complement()
    return seq
    
from multiprocessing import Pool
import itertools

def get_off_targets(cdna_seq, sgRNA_seq, max_mismatches, min_consecutive_matches, num_processes=None,num_threads=None):
    off_targets = []
    if num_processes:
        with Pool(num_processes) as pool:
            for sgRNA, cdna in tqdm(itertools.product(sgRNA_seq, cdna_seq), total=len(sgRNA_seq)*len(cdna_seq), desc='sgRNA'):
                forward_result = pool.apply_async(pairwise_local_align, args=(cdna, sgRNA, max_mismatches, min_consecutive_matches))
                reverse_result = pool.apply_async(pairwise_local_align, args=(cdna, reverse_complement(sgRNA), max_mismatches, min_consecutive_matches))
                forward_result = forward_result.get()
                reverse_result = reverse_result.get()
                if forward_result:
                    off_targets.append({'sgRNA': sgRNA.id, 'cdna': cdna.id, 'strand': '+', **forward_result})
                if reverse_result:
                    off_targets.append({'sgRNA': sgRNA.id, 'cdna': cdna.id, 'strand': '-', **reverse_result})
        return pd.DataFrame(off_targets)
    elif num_threads:
        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            for sgRNA, cdna in tqdm(itertools.product(sgRNA_seq, cdna_seq), total=len(sgRNA_seq)*len(cdna_seq), desc='sgRNA'):
                forward_result = executor.submit(pairwise_local_align, cdna, sgRNA, max_mismatches, min_consecutive_matches)
                reverse_result = executor.submit(pairwise_local_align, cdna, reverse_complement(sgRNA), max_mismatches, min_consecutive_matches)
                forward_result = forward_result.result()
                reverse_result = reverse_result.result()
                if forward_result:
                    off_targets.append({'sgRNA': sgRNA.id, 'cdna': cdna.id, 'strand': '+', **forward_result})
                if reverse_result:
                    off_targets.append({'sgRNA': sgRNA.id, 'cdna': cdna.id, 'strand': '-', **reverse_result})
        return pd.DataFrame(off_targets)
    else:
        for sgRNA, cdna in tqdm(itertools.product(sgRNA_seq, cdna_seq), total=len(sgRNA_seq)*len(cdna_seq), desc='sgRNA'):
            forward_result = pairwise_local_align(cdna, sgRNA, max_mismatches, min_consecutive_matches)
            reverse_result = pairwise_local_align(cdna, reverse_complement(sgRNA), max_mismatches, min_consecutive_matches)
            if forward_result:
                off_targets.append({'sgRNA': sgRNA.id, 'cdna': cdna.id, 'strand': '+', **forward_result})
            if reverse_result:
                off_targets.append({'sgRNA': sgRNA.id, 'cdna': cdna.id, 'strand': '-', **reverse_result})
        return pd.DataFrame(off_targets)

In [13]:
# print(pairwise_local_align(target_seq[17], sgRNA_seq[3]))
# for i in pairwise2.align.globalms(
#     target_seq[17], sgRNA_seq[3], 
#     2, -1, -8, -0.5, 
#     one_alignment_only=True, gap_char='-', force_generic=True
#     ):
#     print(i)
    
# for i in pairwise2.align.globalms(
#     target_seq[17], reverse_complement(sgRNA_seq[3]), 
#     2, -1, -8, -0.5, 
#     one_alignment_only=True, gap_char='-', force_generic=True
#     ):
#     print(i)
    
# for i in pairwise2.align.globalxx(
#     target_seq[17], sgRNA_seq[3]
#     ):
#     print(i)
#     break
    
# for i in pairwise2.align.globalxx(
#     target_seq[17], reverse_complement(sgRNA_seq[3])
#     ):
#     print(i)
#     break

In [14]:
# 使用两两比对获得比对信息
# max_mismatches = 6
# min_consecutive_matches = 21
# threads = 10
# off_targets = get_off_targets(
#     cdna_seq=cdna_seq, 
#     sgRNA_seq=sgRNA_seq, 
#     max_mismatches=max_mismatches, 
#     min_consecutive_matches=min_consecutive_matches, 
#     num_processes=threads
#     )

In [15]:
import os

In [20]:
def split_sgrna(sgrna_path, out_path, strand="both"):
    """
    将sgRNA的FASTA序列切分为一条序列一个文件，并取反向互补序列
    """
    sglist = []
    sgrna_seq = SeqIO.parse(sgrna_path, 'fasta')
    for record in sgrna_seq:
        record.seq = record.seq.upper()
        sgrna_name = record.id.split(":")[0]
        swater = f'{out_path}_{sgrna_name}.water'

        fwd_fa = f'{out_path}_water_sg_fwd_{sgrna_name}.fasta'
        fwd_water = f'{out_path}_water_sg_fwd_{sgrna_name}.water'
        fmesg = [sgrna_name, record.seq, fwd_fa, fwd_water, swater]

        rev_fa = f'{out_path}_water_sg_rev_{sgrna_name}.fasta'
        rev_water = f'{out_path}_water_sg_rev_{sgrna_name}.water'
        rmesg = [sgrna_name, record.seq.reverse_complement(), rev_fa, rev_water, swater]

        time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        if strand == "fwd":
            sglist.append(fmesg)
            record.seq = record.seq
            SeqIO.write(record, fwd_fa, "fasta")
            print(f'{time} <PARAM>  {fwd_fa} {fmesg[1]}')
        elif strand == "rev":
            sglist.append(rmesg)
            record.seq = record.seq.reverse_complement()
            SeqIO.write(record, rev_fa, "fasta")
            print(f'{time} <PARAM>  {rev_fa} {rmesg[2]}')
        elif strand == "both":
            sglist.append(fmesg)
            SeqIO.write(record, fwd_fa, "fasta")
            print(f'{time} <PARAM>  {fwd_fa} {fmesg[1]}')
            sglist.append(rmesg)
            record.seq = record.seq.reverse_complement()
            SeqIO.write(record, rev_fa, "fasta")
            print(f'{time} <PARAM>  {rev_fa} {rmesg[2]}')
    return sglist

def get_water_alignment(sgrna_path, cdna_path, out_path, gapopen=10, gapextend=0.5):
    """
    将切分的sgRNA序列与转录本比对
    """
    wcline = WaterCommandline(
        gapopen=gapopen,
        gapextend=gapextend,
        asequence=sgrna_path,
        bsequence=cdna_path,
        outfile=out_path
    )
    print(wcline)
    stdout, stderr = wcline()
    print(stdout + stderr)
    return out_path


def pool_get_water_alignment(sgrna_list, cdna_path, gapopen=10, gapextend=0.5, threads=2, realign=False):
    pool = Pool(processes=threads)
    if realign:
        params = [(record[2], cdna_path, record[3], gapopen, gapextend) for record in sgrna_list if not os.path.exists(record[3])]
    else:
        params = [(record[2], cdna_path, record[3], gapopen, gapextend) for record in sgrna_list]
    results = []
    with tqdm(desc='Aligning', total=len(params)) as pbar:
        for param in params:
            result = pool.apply_async(func=get_water_alignment, args=param, callback=lambda x: pbar.update())
        results.append(result.get())
        pool.close()
        pool.join()
    time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'{time} <OUTPUT> {results}')
    return results

In [17]:
import datetime
# 分割sgRNA到fasta文件
out_path = "./Predictions"
strand = 'both'
sgRNA_list = split_sgrna(
    sgrna_path=sgRNA_path,
    out_path=out_path,
    strand=strand
)
print(sgRNA_list)

2023-04-11 17:24:54 <PARAM>  ./Predictions_water_sg_fwd_sgRNA1.fasta GAATGTAGCCATCCATCCTTGTCAA
2023-04-11 17:24:54 <PARAM>  ./Predictions_water_sg_rev_sgRNA1.fasta ./Predictions_water_sg_rev_sgRNA1.fasta
2023-04-11 17:24:54 <PARAM>  ./Predictions_water_sg_fwd_sgRNA2.fasta ATGTTGGACTCCTCAGTGGGCACACACTCC
2023-04-11 17:24:54 <PARAM>  ./Predictions_water_sg_rev_sgRNA2.fasta ./Predictions_water_sg_rev_sgRNA2.fasta
2023-04-11 17:24:54 <PARAM>  ./Predictions_water_sg_fwd_sgRNA3.fasta GTGCTGTAGGAAGCTCATCTCTCCTATGTG
2023-04-11 17:24:54 <PARAM>  ./Predictions_water_sg_rev_sgRNA3.fasta ./Predictions_water_sg_rev_sgRNA3.fasta
[['sgRNA1', Seq('GAATGTAGCCATCCATCCTTGTCAA'), './Predictions_water_sg_fwd_sgRNA1.fasta', './Predictions_water_sg_fwd_sgRNA1.water', './Predictions_sgRNA1.water'], ['sgRNA1', Seq('TTGACAAGGATGGATGGCTACATTC'), './Predictions_water_sg_rev_sgRNA1.fasta', './Predictions_water_sg_rev_sgRNA1.water', './Predictions_sgRNA1.water'], ['sgRNA2', Seq('ATGTTGGACTCCTCAGTGGGCACACACTCC'), './

In [21]:
from Bio.Emboss.Applications import WaterCommandline
# 批量比对获取比对结果文件列表
gapopen = 10
gapextend = 0.5
threads = 10
pool_get_water_alignment(
    sgrna_list=sgRNA_list,
    cdna_path=cdna_path,
    gapopen=gapopen,
    gapextend=gapextend,
    threads=threads
)

Aligning:   0%|          | 0/6 [00:00<?, ?it/s]

water -outfile=./Predictions_water_sg_rev_sgRNA1.water -asequence=./Predictions_water_sg_rev_sgRNA1.fasta -bsequence=./Homo_sapiens.GRCh38.cdna.all.fa -gapopen=10 -gapextend=0.5water -outfile=./Predictions_water_sg_fwd_sgRNA2.water -asequence=./Predictions_water_sg_fwd_sgRNA2.fasta -bsequence=./Homo_sapiens.GRCh38.cdna.all.fa -gapopen=10 -gapextend=0.5water -outfile=./Predictions_water_sg_fwd_sgRNA3.water -asequence=./Predictions_water_sg_fwd_sgRNA3.fasta -bsequence=./Homo_sapiens.GRCh38.cdna.all.fa -gapopen=10 -gapextend=0.5water -outfile=./Predictions_water_sg_rev_sgRNA2.water -asequence=./Predictions_water_sg_rev_sgRNA2.fasta -bsequence=./Homo_sapiens.GRCh38.cdna.all.fa -gapopen=10 -gapextend=0.5water -outfile=./Predictions_water_sg_fwd_sgRNA1.water -asequence=./Predictions_water_sg_fwd_sgRNA1.fasta -bsequence=./Homo_sapiens.GRCh38.cdna.all.fa -gapopen=10 -gapextend=0.5water -outfile=./Predictions_water_sg_rev_sgRNA3.water -asequence=./Predictions_water_sg_rev_sgRNA3.fasta -bsequenc

['./Predictions_water_sg_rev_sgRNA3.water']

In [22]:
from Bio import AlignIO

In [23]:
def cal_mismatch_and_gap(cdna, sgrna, sgrna_len):
    cdna_seq = cdna.seq
    sgrna_seq = sgrna.seq
    unalign = 0
    match = 0
    max_seed = []
    seed = 0
    for cdna_base, sgrna_base in zip(cdna_seq, sgrna_seq):
        if cdna_base != sgrna_base:
            unalign += 1
            max_seed.append(seed)
            seed = 0
        else:
            seed += 1
            match += 1
    max_seed.append(seed)
    unmatch = sgrna_len - match
    max_seed = max(max_seed)
    return match, unmatch, max_seed


def filter_water_result(water_out_path, sgrna_len, min_match, max_unalign, sgrna_id):
    water = list(AlignIO.parse(water_out_path, 'emboss'))
    water_out = []
    with tqdm(total=len(water), desc=f'Filtering {sgrna_id}') as pbar:
        for record in water:
            sgrna, cdna = record
            match, unmatch, max_seed_len = cal_mismatch_and_gap(cdna, sgrna, sgrna_len)
            align_len = record.get_alignment_length()
            if max_seed_len >= 8 and match >= sgrna_len - max_unalign and match >= min_match and sgrna_len + max_unalign >= align_len:
                mesg = [sgrna_id, cdna.id, sgrna.seq, cdna.seq, sgrna_len, match, unmatch, max_seed_len, align_len]
                water_out.append(mesg)
            pbar.update(1)
    time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'{time} <RESULT> {sgrna_id} got {len(water_out)} results')
    return water_out


def pool_filter_water_result(sgrna_list, min_match, max_unalign, threads=2):
    pool = Pool(processes=threads)
    results_list = []
    for record in tqdm(sgrna_list, desc='Filtering result'):
        water_path = record[3]
        sgrna_len = len(record[1])
        sgrna_id = record[0]
        result = pool.apply_async(filter_water_result, args=(water_path, sgrna_len, min_match, max_unalign, sgrna_id))
        results_list.append(result.get())
    pool.close()
    pool.join()
    time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'{time} <RESULT> got {len(results_list)} different sgRNA result')
    return results_list

In [24]:
min_match = 21
max_unmatch = 6
threads = 5
flt_list = pool_filter_water_result(
    sgrna_list=sgRNA_list,
    min_match=min_match,
    max_unalign=max_unmatch,
    threads=threads
)

Filtering result:   0%|          | 0/6 [00:00<?, ?it/s]

2023-04-11 17:37:08 <RESULT> sgRNA1 got 319 results
2023-04-11 17:37:18 <RESULT> sgRNA1 got 535 results
2023-04-11 17:37:28 <RESULT> sgRNA2 got 385 results
2023-04-11 17:37:38 <RESULT> sgRNA2 got 424 results
2023-04-11 17:37:49 <RESULT> sgRNA3 got 379 results


Filtering sgRNA3:   0%|          | 0/191887 [00:00<?, ?it/s]

2023-04-11 17:37:59 <RESULT> sgRNA3 got 610 results
2023-04-11 17:37:59 <RESULT> got 6 different sgRNA result


In [25]:
def merge_to_dataframe(multi_list):
    alist = []
    for res in multi_list:
        alist.extend(res)
    df = pd.DataFrame(
        alist,
        columns=['sgRNA_ID', 'Transcript_ID', 'sgRNA_align', 
                 'Transcript_align', 'sgRNA_len', 'Match', 'Unmatch',
                 'Max_seed', 'Alignment_length']
    )
    df.drop_duplicates()
    df.sort_values(
        by=["sgRNA_ID", "Unmatch", "Transcript_ID"], 
        ascending=[False, True, False], 
        inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [26]:

flt_df = merge_to_dataframe(flt_list)
flt_file = f'{out_path}_water_fltM{min_match}U{max_unmatch}.result.txt'
flt_df.to_csv(flt_file, sep="\t", index=False, encoding='utf-8')
flt_df

,sgRNA_ID,Transcript_ID,sgRNA_align,Transcript_align,sgRNA_len,Match,Unmatch,Max_seed,Alignment_length
0,sgRNA3,ENST00000672860.2,"(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...","(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...",30,30,0,30,30
1,sgRNA3,ENST00000523950.5,"(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...","(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...",30,30,0,30,30
2,sgRNA3,ENST00000523873.5,"(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...","(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...",30,30,0,30,30
3,sgRNA3,ENST00000523125.5,"(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...","(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...",30,30,0,30,30
4,sgRNA3,ENST00000520948.5,"(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...","(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...",30,30,0,30,30
...,...,...,...,...,...,...,...,...,...
2647,sgRNA1,ENST00000232975.8,"(T, T, G, A, C, A, A, G, G, A, T, G, -, -, G, ...","(T, T, G, A, C, A, A, A, A, A, T, G, C, T, G, ...",25,21,4,11,25
2648,sgRNA1,ENST00000225742.13,"(T, T, G, A, C, A, A, G, G, A, -, -, T, G, G, ...","(T, G, G, A, C, A, A, G, G, A, G, C, T, G, T, ...",25,21,4,8,30
2649,sgRNA1,ENST00000219789.11,"(G, A, A, T, G, T, A, G, C, C, A, T, C, C, A, ...","(G, A, A, -, G, A, A, G, C, C, A, T, C, C, A, ...",25,21,4,9,30
2650,sgRNA1,ENST00000201963.3,"(T, T, G, A, C, A, A, G, G, A, T, G, -, -, -, ...","(T, T, T, A, C, A, T, G, T, A, T, G, A, T, G, ...",25,21,4,8,31


In [27]:
def anno_transcript(data=None, data_path=None, trans_anno=None, cdna_path=None):
    if data_path and not data:
        data =  pd.read_csv(data_path, sep="\t")
    if cdna_path and not trans_anno:
        cdna_fa = SeqIO.parse(cdna_path, "fasta")
        trans_anno = getTransformat(cdna_fa)
    data = pd.merge(data, trans_anno, how='inner', on='Transcript_ID')
    data.drop_duplicates(inplace=True)
    data.sort_values(by=["sgRNA_ID", 'Max_seed',"Match"], ascending=[False, False, False], inplace=True)
    data.reset_index(drop=True, inplace=True)
    return data

In [28]:
anno_df = anno_transcript(
    data=flt_df, 
    data_path=None, 
    trans_anno=cdna_anno, 
    cdna_path=None)
anno_file = f'{out_path}_water_fltM{min_match}U{max_unmatch}.result.annotated.txt'
anno_df.to_csv(anno_file, sep="\t", index=False, encoding='utf-8')
anno_df

,sgRNA_ID,Transcript_ID,sgRNA_align,Transcript_align,sgRNA_len,Match,Unmatch,Max_seed,Alignment_length,Gene_ID,Gene_Symbol,Gene_Position,Transcript_Biotype,Transcript_Length
0,sgRNA3,ENST00000672860.2,"(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...","(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...",30,30,0,30,30,ENSG00000112715.24,VEGFA,6:43770213-43786487:1,protein_coding,3535
1,sgRNA3,ENST00000523950.5,"(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...","(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...",30,30,0,30,30,ENSG00000112715.24,VEGFA,6:43771209-43784902:1,protein_coding,954
2,sgRNA3,ENST00000523873.5,"(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...","(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...",30,30,0,30,30,ENSG00000112715.24,VEGFA,6:43771209-43784609:1,protein_coding,784
3,sgRNA3,ENST00000523125.5,"(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...","(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...",30,30,0,30,30,ENSG00000112715.24,VEGFA,6:43771247-43784562:1,protein_coding,541
4,sgRNA3,ENST00000520948.5,"(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...","(C, A, C, A, T, A, G, G, A, G, A, G, A, T, G, ...",30,30,0,30,30,ENSG00000112715.24,VEGFA,6:43771083-43784949:1,protein_coding,1199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2647,sgRNA1,ENST00000244204.11,"(T, T, G, A, C, A, A, G, G, A, T, -, -, -, -, ...","(T, T, G, A, -, A, A, G, G, T, T, A, T, G, A, ...",25,21,4,8,30,ENSG00000124357.13,NAGK,2:71068648-71079808:1,protein_coding,2371
2648,sgRNA1,ENST00000233055.9,"(T, G, A, C, A, A, G, G, A, T, G, -, -, -, -, ...","(T, G, A, C, A, A, G, G, -, T, G, C, A, G, T, ...",25,21,4,8,30,ENSG00000085449.15,WDFY1,2:223875348-223945335:-1,protein_coding,4607
2649,sgRNA1,ENST00000225742.13,"(T, T, G, A, C, A, A, G, G, A, -, -, T, G, G, ...","(T, G, G, A, C, A, A, G, G, A, G, C, T, G, T, ...",25,21,4,8,30,ENSG00000108604.16,SMARCD2,17:63832604-63842466:-1,protein_coding,1800
2650,sgRNA1,ENST00000201963.3,"(T, T, G, A, C, A, A, G, G, A, T, G, -, -, -, ...","(T, T, T, A, C, A, T, G, T, A, T, G, A, T, G, ...",25,21,4,8,31,ENSG00000088305.18,DNMT3B,20:32779852-32809356:1,protein_coding,4255


In [29]:
import subprocess
import gzip


def remove_tmpfile(sgrna_list):
    gzfile = []
    time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'{time} <RESULT> Creating gzipfile for result and remove tmpfile.')
    with tqdm(total=len(sgrna_list), desc='Removing tmpfile') as pbar:
        for record in sgrna_list:
            sgrna_fa = record[2]
            water_out = record[3]
            gzipout = f'{water_out}.gz'
            if os.path.exists(gzipout):
                gzfile.append(gzipout)
                pbar.update(1) 
                continue
            with open(water_out, "rb") as f_in:
                f_value = f_in.read()
            f_in.close()

            with gzip.open(gzipout, "wb", 9) as gzip_out:
                gzip_out.write(memoryview(f_value))
            gzip_out.close()
            try:
                os.remove(sgrna_fa)
                os.remove(water_out)
                gzfile.append(gzipout)
            except:
                continue
            pbar.update(1)        
    return gzfile

def tar_files(file_list, output_file):
    try:
        subprocess.run(['tar', '-czvf', output_file] + file_list)
        for file in file_list:
            os.remove(file)
    except:
        pass
    

In [30]:
tar_file = f'{out_path}_water_fltM{min_match}U{max_unmatch}.result.tar.gz'
gz_files = remove_tmpfile(sgrna_list=sgRNA_list)
print(gz_files)
tar_files(gz_files, tar_file)
print(tar_file)

2023-04-11 17:37:59 <RESULT> Creating gzipfile for result and remove tmpfile.


Removing tmpfile:   0%|          | 0/6 [00:00<?, ?it/s]

['./Predictions_water_sg_fwd_sgRNA1.water.gz', './Predictions_water_sg_rev_sgRNA1.water.gz', './Predictions_water_sg_fwd_sgRNA2.water.gz', './Predictions_water_sg_rev_sgRNA2.water.gz', './Predictions_water_sg_fwd_sgRNA3.water.gz', './Predictions_water_sg_rev_sgRNA3.water.gz']
./Predictions_water_sg_fwd_sgRNA1.water.gz
./Predictions_water_sg_rev_sgRNA1.water.gz
./Predictions_water_sg_fwd_sgRNA2.water.gz
./Predictions_water_sg_rev_sgRNA2.water.gz
./Predictions_water_sg_fwd_sgRNA3.water.gz
./Predictions_water_sg_rev_sgRNA3.water.gz
./Predictions_water_fltM21U6.result.tar.gz


In [31]:
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from tqdm import tqdm
import os

def blast_local_align(seq1, seq2, max_mismatches=None, min_consecutive_matches=None):
    seq1_file = 'seq1.fasta'
    seq2_file = 'seq2.fasta'
    with open(seq1_file, 'w') as f:
        f.write(f'>{seq1.id}\n{seq1.seq}\n')
    with open(seq2_file, 'w') as f:
        f.write(f'>{seq2.id}\n{seq2.seq}\n')
    blastn_cline = NcbiblastnCommandline(query=seq1_file, subject=seq2_file, outfmt=5)
    stdout, stderr = blastn_cline()
    blast_records = NCBIXML.parse(stdout)
    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                cigar = ''
                seq1 = Seq(hsp.query)
                seq2 = Seq(hsp.sbjct)
                for i in range(len(seq1)):
                    if seq1[i] == seq2[i]:
                        cigar += '|'
                    elif seq1[i] == '-':
                        cigar += 'I'
                    elif seq2[i] == '-':
                        cigar += 'D'
                    else:
                        cigar += 'X'
                matches = cigar.count('|')
                mismatches = cigar.count('X') + cigar.count('I') + cigar.count('D')
                max_consecutive_matches = max([len(x) for x in re.split('[DXI]', cigar)])
                if (max_mismatches and mismatches > max_mismatches) or (min_consecutive_matches and max_consecutive_matches < min_consecutive_matches):
                    return None
                else:
                    return {'matches': matches, 'mismatches': mismatches, 'start': hsp.query_start, 'end': hsp.query_end,
                            'max_consecutive_matches': max_consecutive_matches, 
                            'seq1':seq1[hsp.query_start-1:hsp.query_end], 'seq2':seq2[hsp.query_start-1:hsp.query_end], 'cigar':cigar}
    os.remove(seq1_file)
    os.remove(seq2_file)
    return None


In [32]:
# import subprocess

# # 定义sgRNA序列和全cDNA序列的文件路径
# sgRNA_file = "/mnt/d/download/human_genome/hg38/Homo_sapiens.GRCh38.cdna.all.fa"
# cDNA_file = "./sgRNA_sequence.fasta"

# # 定义blastn命令行参数
# blastn_args = ["blastn", "-query", sgRNA_file, "-task", "blastn-short",
#                "-subject", cDNA_file, "-outfmt", "0", "-evalue", "0.5",
#                "-gapopen", "7", "-gapextend", "2"]

# # 运行blastn命令行并将结果保存至文件
# with open("blast_result.txt", "w") as output_file:
#     subprocess.run(blastn_args, stdout=output_file)